In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
try:
    trp_revenue = pd.read_excel('TRP_VENITURI.xlsx')
    permits = pd.read_excel('building_permitsVALORI.xlsx')
    constr = pd.read_excel('Construction_index_anual.xlsx')
    capex = pd.read_excel('ch_cu_investitii.xlsx')
except FileNotFoundError as e:
    print(f"Error: Could not find file. {e}")
    exit()

In [3]:
permits = permits[['Year', 'Number of Building Permits']].copy()
permits.columns = ['Year', 'Total_Permits']
permits_annual = permits.set_index('Year').sort_index()
permits_annual['Permits_Lag1'] = permits_annual['Total_Permits'].shift(1)

In [4]:
constr_df = constr.set_index('Year').sort_index()
if '%change' in constr_df.columns:
    constr_df = constr_df.rename(columns={'%change': 'Construction_Index'})
else:
    constr_df.columns = ['Construction_Index']
capex_df = capex.set_index('Year').sort_index()
capex_df = capex_df.iloc[:, 0].to_frame(name='Govt_Capex')
col_revenue_name = 'Millions RON'
if col_revenue_name in trp_revenue.columns:
    trp_revenue[col_revenue_name] = (
        trp_revenue[col_revenue_name]
        .astype(str)
        .str.replace(' ', '')
        .astype(float)
    )
    revenue_df = trp_revenue[['Year', col_revenue_name]].set_index('Year').sort_index()
else:
    print(f"Column '{col_revenue_name}' not found in TRP_VENITURI.xlsx")
    exit()

In [5]:
df = revenue_df.join(permits_annual[['Permits_Lag1']], how='inner')
df = df.join(constr_df, how='inner')
df = df.join(capex_df, how='inner')
df_clean = df.dropna()
df_model = df_clean.drop(index=2021, errors='ignore')
X = df_model[['Permits_Lag1', 'Construction_Index', 'Govt_Capex']]
y = df_model['Millions RON']

In [6]:
model = LinearRegression()
model.fit(X, y)
r2 = model.score(X, y)

In [7]:
try:
    val_permits_lag = permits_annual.loc[2024, 'Total_Permits']
    val_index_2025 = constr_df.loc[2025, 'Construction_Index']
    val_capex_2025 = capex_df.loc[2025, 'Govt_Capex']
    X_2025 = pd.DataFrame([[val_permits_lag, val_index_2025, val_capex_2025]],
                          columns=['Permits_Lag1', 'Construction_Index', 'Govt_Capex'])
    pred_2025 = model.predict(X_2025)[0]
    pred_msg = f"{pred_2025:,.0f} RON"
except KeyError as e:
    pred_msg = f"Cannot predict 2025: Missing data for {e}"
permits_2025 = permits_annual.loc[2025, 'Total_Permits'] if 2025 in permits_annual.index else 0
permits_2024 = permits_annual.loc[2024, 'Total_Permits']
permits_change_pct = ((permits_2025 - permits_2024) / permits_2024) * 100
trend_desc = "INCREASED" if permits_change_pct > 0 else "DECREASED"

In [8]:
print("=" * 60)
print(f"TERAPLAST REVENUE MODEL (RÂ² = {r2:.2f})")
print("=" * 60)
print(f"2025 Market Drivers:")
print(f"  - Construction Volume: {val_index_2025:.1f}")
print(f"  - Permits (2024 Lag):  {val_permits_lag:,.0f}")
print(f"  - Govt Investment:     {val_capex_2025} Billion RON")
print("-" * 60)
print(f"PREDICTION FOR 2025 REVENUE: {pred_msg}")
print("-" * 60)
print(f"Future Signal (for 2026):")
print(f"  Permits in 2025 have {trend_desc} by {abs(permits_change_pct):.1f}% vs 2024.")
if trend_desc == "DECREASED":
    print("  WARNING: Fewer permits in 2025 suggests a potential revenue SLOWDOWN in 2026.")
else:
    print("  POSITIVE: More permits in 2025 suggests strong demand continuing into 2026.")
print("=" * 60)

TERAPLAST REVENUE MODEL (RÂ² = 0.97)
2025 Market Drivers:
  - Construction Volume: 176.6
  - Permits (2024 Lag):  36,206
  - Govt Investment:     138.2 Billion RON
------------------------------------------------------------
PREDICTION FOR 2025 REVENUE: 757,795,457 RON
------------------------------------------------------------
Future Signal (for 2026):
  Permits in 2025 have INCREASED by 2.9% vs 2024.
  POSITIVE: More permits in 2025 suggests strong demand continuing into 2026.
